In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv(r'C:\Users\DUU\Desktop\rileys_data.csv') 

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 30 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   AIRLINE           1048575 non-null  object 
 1   ORIGIN            1048575 non-null  object 
 2   ORIGIN_CITY       1048575 non-null  object 
 3   DEST              1048575 non-null  object 
 4   DEST_CITY         1048575 non-null  object 
 5   DEP_DELAY         1021438 non-null  float64
 6   CRS_ARR_TIME      1048575 non-null  int64  
 7   ARR_DELAY         1018520 non-null  float64
 8   CANCELLED         1048575 non-null  int64  
 9   DIVERTED          1048575 non-null  int64  
 10  DISTANCE          1048575 non-null  int64  
 11  ori_TMIN          1048575 non-null  int64  
 12  ori_TMAX          1048575 non-null  int64  
 13  ori_SNOW          1048575 non-null  int64  
 14  ori_SNWD          1048575 non-null  int64  
 15  ori_AWND          1048575 non-null  int64  
 16  

In [3]:
df.drop(['DEP_DELAY', 'CRS_ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'CRS_DEP_Datetime'], axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 24 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   AIRLINE      1048575 non-null  object
 1   ORIGIN       1048575 non-null  object
 2   ORIGIN_CITY  1048575 non-null  object
 3   DEST         1048575 non-null  object
 4   DEST_CITY    1048575 non-null  object
 5   DISTANCE     1048575 non-null  int64 
 6   ori_TMIN     1048575 non-null  int64 
 7   ori_TMAX     1048575 non-null  int64 
 8   ori_SNOW     1048575 non-null  int64 
 9   ori_SNWD     1048575 non-null  int64 
 10  ori_AWND     1048575 non-null  int64 
 11  ori_PRCP     1048575 non-null  int64 
 12  dest_TMIN    1048575 non-null  int64 
 13  dest_TMAX    1048575 non-null  int64 
 14  dest_SNOW    1048575 non-null  int64 
 15  dest_SNWD    1048575 non-null  int64 
 16  dest_AWND    1048575 non-null  int64 
 17  dest_PRCP    1048575 non-null  int64 
 18  15_DELAYED   1048575 n

In [5]:
# Define a function to replace infrequent categories with 'Other'
def replace_infrequent_categories(df, categorical_cols, threshold=0.2, other_label='Other'):
    for col in categorical_cols:
        counts = df[col].value_counts(normalize=True)
        infrequent_categories = counts[counts < threshold].index
        df[col] = df[col].replace(infrequent_categories, other_label)
    return df

# Define threshold for infrequent categories (adjust as needed)
threshold = 0.2
print("w")



w


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 24 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   AIRLINE      1048575 non-null  object
 1   ORIGIN       1048575 non-null  object
 2   ORIGIN_CITY  1048575 non-null  object
 3   DEST         1048575 non-null  object
 4   DEST_CITY    1048575 non-null  object
 5   DISTANCE     1048575 non-null  int64 
 6   ori_TMIN     1048575 non-null  int64 
 7   ori_TMAX     1048575 non-null  int64 
 8   ori_SNOW     1048575 non-null  int64 
 9   ori_SNWD     1048575 non-null  int64 
 10  ori_AWND     1048575 non-null  int64 
 11  ori_PRCP     1048575 non-null  int64 
 12  dest_TMIN    1048575 non-null  int64 
 13  dest_TMAX    1048575 non-null  int64 
 14  dest_SNOW    1048575 non-null  int64 
 15  dest_SNWD    1048575 non-null  int64 
 16  dest_AWND    1048575 non-null  int64 
 17  dest_PRCP    1048575 non-null  int64 
 18  15_DELAYED   1048575 n

In [7]:
scale_pos_weight = 3.557

In [8]:
categorical_columns = ['AIRLINE', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY']
numerical_columns = df.columns.difference(categorical_columns + ['15_DELAYED'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_columns),  # Apply no transformation to numerical columns
        ('cat', OneHotEncoder(), categorical_columns)  # Apply OneHotEncoder to categorical columns
    ]
)

X = preprocessor.fit_transform(df.drop('15_DELAYED', axis=1))
y = df['15_DELAYED']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight)
xgb_classifier.fit(X_train, y_train)

y_pred = xgb_classifier.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.69      0.77    163691
           1       0.37      0.65      0.47     46024

    accuracy                           0.68    209715
   macro avg       0.62      0.67      0.62    209715
weighted avg       0.76      0.68      0.70    209715

Accuracy: 0.6795841976015068


In [10]:
##This Entire part is for the grid search to find the best parameter but the results not good. Skip this part
import pandas as pd
import xgboost as xgb
import time
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score


# Sample a subset of your data for quicker execution (e.g., using 20% of the data)
X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.8, random_state=42)

# Split the subset of data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

# Define your parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'colsample_bytree': [0.3, 0.7],
    'subsample': [0.7, 1],
    'gamma': [0, 1, 5]
}

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Initialize RandomizedSearchCV with the XGBoost classifier and parameter grid
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_grid,
    n_iter=10,  # number of parameter settings that are sampled
    scoring='roc_auc',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Start timing the grid search
start_time = time.time()

# Fit the RandomizedSearchCV to the training data with early stopping
random_search.fit(
    X_train,
    y_train,
    early_stopping_rounds=10,
    eval_set=[(X_test, y_test)],
    verbose=True
)

# Calculate and print the total time taken
end_time = time.time()
total_time = end_time - start_time
print(f"Total Time for Randomized Search: {total_time} seconds")

# Print the best parameters and the best score
print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)

# Make predictions with the best estimator
best_estimator = random_search.best_estimator_
y_pred = best_estimator.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


X:\Python 3.11\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.52411
[1]	validation_0-logloss:0.52193
[2]	validation_0-logloss:0.52005
[3]	validation_0-logloss:0.51722
[4]	validation_0-logloss:0.51389
[5]	validation_0-logloss:0.51120
[6]	validation_0-logloss:0.50890
[7]	validation_0-logloss:0.50677
[8]	validation_0-logloss:0.50576
[9]	validation_0-logloss:0.50491
[10]	validation_0-logloss:0.50212
[11]	validation_0-logloss:0.49982
[12]	validation_0-logloss:0.49820
[13]	validation_0-logloss:0.49680
[14]	validation_0-logloss:0.49622
[15]	validation_0-logloss:0.49510
[16]	validation_0-logloss:0.49368
[17]	validation_0-logloss:0.49240
[18]	validation_0-logloss:0.49150
[19]	validation_0-logloss:0.49060
[20]	validation_0-logloss:0.48984
[21]	validation_0-logloss:0.48912
[22]	validation_0-logloss:0.48846
[23]	validation_0-logloss:0.48786
[24]	validation_0-logloss:0.48707
[25]	validation_0-logloss:0.48639
[26]	validation_0-logloss:0.48588
[27]	validation_0-logloss:0.48542
[28]	validation_0-logloss:0.48503
[29]	validation_0-loglos